In [ ]:
import numpy as np
from tensorflow.keras import layers, models, optimizers, metrics
from matplotlib import pyplot as plt
import pandas as pd
import tensorflow as tf

In [ ]:
normal = pd.read_csv("../ecg/ptbdb_normal.csv")
abnormal = pd.read_csv("../ecg/ptbdb_abnormal.csv")

In [ ]:
normal = normal.values[:,:-1]
abnormal = abnormal.values[:,:-1]

In [ ]:
normal = normal - 0.5
abnormal = abnormal - 0.5

In [ ]:
def show_ecg(signal):
    plt.figure(figsize=(12, 1))
    plt.plot(np.arange(signal.size), signal, c="black")
    plt.show()

In [ ]:
show_ecg(normal[10])

In [ ]:
show_ecg(abnormal[10])

In [ ]:
normal.shape, abnormal.shape

In [ ]:
X = np.concatenate((normal, abnormal))
y = np.concatenate((np.zeros(normal.shape[0]), np.ones(abnormal.shape[0])))

In [ ]:
X = np.pad(X, ((0, 0), (0, 5)), mode='edge')

In [ ]:
X = np.expand_dims(X, axis=2)

In [ ]:
X.shape, y.shape

In [ ]:
p = np.random.permutation(y.size - 1)
X = X[p]
y = y[p]

In [ ]:
from __future__ import print_function, division

from tensorflow.keras.datasets import mnist
from tensorflow.keras.layers import Input, Dense, Reshape, Flatten, Dropout, multiply
from tensorflow.keras.layers import BatchNormalization, Activation, Embedding, ZeroPadding1D
from tensorflow.keras.layers import LeakyReLU
from tensorflow.keras.layers import UpSampling1D, Conv1D
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.optimizers import Adam

import matplotlib.pyplot as plt

import numpy as np

class ACGAN():
    def __init__(self):
        # Input shape
        self.img_rows = 192
        self.channels = 1
        self.img_shape = (self.img_rows, self.channels)
        self.num_classes = 2
        self.latent_dim = 100

        optimizer = Adam(0.0002, 0.5)
        losses = ['binary_crossentropy', 'sparse_categorical_crossentropy']

        # Build and compile the discriminator
        self.discriminator = self.build_discriminator()
        self.discriminator.compile(loss=losses,
            optimizer=optimizer,
            metrics=['accuracy'])

        # Build the generator
        self.generator = self.build_generator()

        # The generator takes noise and the target label as input
        # and generates the corresponding digit of that label
        noise = Input(shape=(self.latent_dim,))
        label = Input(shape=(1,))
        img = self.generator([noise, label])

        # For the combined model we will only train the generator
        self.discriminator.trainable = False

        # The discriminator takes generated image as input and determines validity
        # and the label of that image
        valid, target_label = self.discriminator(img)

        # The combined model  (stacked generator and discriminator)
        # Trains the generator to fool the discriminator
        self.combined = Model([noise, label], [valid, target_label])
        self.combined.compile(loss=losses,
            optimizer=optimizer)

    def build_generator(self):

        model = Sequential()

        model.add(Dense(8 * 24, activation="relu", input_dim=self.latent_dim))
        model.add(Reshape((16, 12)))
        model.add(BatchNormalization(momentum=0.8))
        model.add(UpSampling1D(size=3))
        model.add(Conv1D(256, kernel_size=5, padding="same"))
        model.add(Activation("relu"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(UpSampling1D(size=2))
        model.add(Conv1D(128, kernel_size=5, padding="same"))
        model.add(Activation("relu"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(UpSampling1D(size=2))
        model.add(Conv1D(64, kernel_size=5, padding="same"))
        model.add(Activation("relu"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Conv1D(self.channels, kernel_size=3, padding='same'))
        model.add(Activation("tanh"))

        model.summary()

        noise = Input(shape=(self.latent_dim,))
        label = Input(shape=(1,), dtype='int32')
        label_embedding = Flatten()(Embedding(self.num_classes, self.latent_dim)(label))

        model_input = multiply([noise, label_embedding])
        img = model(model_input)

        return Model([noise, label], img)

    def build_discriminator(self):

        model = Sequential()

        model.add(Conv1D(16, kernel_size=7, strides=2, input_shape=self.img_shape, padding="same"))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        model.add(Conv1D(32, kernel_size=7, strides=2, padding="same"))
        model.add(ZeroPadding1D(padding=(0, 1)))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Conv1D(64, kernel_size=7, strides=2, padding="same"))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Conv1D(128, kernel_size=7, strides=1, padding="same"))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))

        model.add(Flatten())
        model.summary()

        img = Input(shape=self.img_shape)

        # Extract feature representation
        features = model(img)

        # Determine validity and label of the image
        validity = Dense(1, activation="sigmoid")(features)
        label = Dense(self.num_classes, activation="softmax")(features)

        return Model(img, [validity, label])

    def train(self, epochs, batch_size=128, sample_interval=50):

        # Load the dataset
        (X_train, y_train) = (X, y)

        # Configure inputs

        y_train = y_train.reshape(-1, 1)

        # Adversarial ground truths
        valid = np.ones((batch_size, 1))
        fake = np.zeros((batch_size, 1))

        for epoch in range(epochs):

            # ---------------------
            #  Train Discriminator
            # ---------------------

            # Select a random batch of images
            idx = np.random.randint(0, X_train.shape[0], batch_size)
            imgs = X_train[idx]

            # Sample noise as generator input
            noise = np.random.normal(0, 1, (batch_size, self.latent_dim))

            # The labels of the digits that the generator tries to create an
            # image representation of
            sampled_labels = np.random.randint(0, 2, (batch_size, 1))

            # Generate a half batch of new images
            gen_imgs = self.generator.predict([noise, sampled_labels])

            # Image labels. 0-9 
            img_labels = y_train[idx]

            # Train the discriminator
            d_loss_real = self.discriminator.train_on_batch(imgs, [valid, img_labels])
            d_loss_fake = self.discriminator.train_on_batch(gen_imgs, [fake, sampled_labels])
            d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

            # ---------------------
            #  Train Generator
            # ---------------------

            # Train the generator
            g_loss = self.combined.train_on_batch([noise, sampled_labels], [valid, sampled_labels])

            # Plot the progress
            if epoch % (sample_interval // 5) == 0:
                print ("%d [D loss: %f, acc.: %.2f%%, op_acc: %.2f%%] [G loss: %f]" % (epoch, d_loss[0], 100*d_loss[3], 100*d_loss[4], g_loss[0]))

            # If at save interval => save generated image samples
            if epoch % sample_interval == 0:
                self.save_model()
                self.sample_images(epoch)

    def sample_images(self, epoch):
        noise = np.random.normal(0, 1, (10, self.latent_dim))
        sampled_labels = np.array(([0] * 5) + ([1] * 5))
        gen_imgs = self.generator.predict([noise, sampled_labels])

        fig, axs = plt.subplots(10, 1)
        for i in range(sampled_labels.size):
            axs[i].plot(gen_imgs[i,:,0], color='black')
        plt.show()

    def save_model(self):
        pass

In [ ]:
acgan = ACGAN()


In [ ]:
acgan.train(epochs=14000, batch_size=32, sample_interval=50)